# 训练分类模型

In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from ultralytics import YOLO
from pathlib import Path

# ===== 你自己改的部分 =====
dataFolderNameList = [
    "insect_split_0.7_0.2_0.1",
    # "insect_split_0.6_0.2_0.2",
    "insect_split_0.5_0.3_0.2",
    # "insect_split_0.5_0.2_0.3",
    "insect_split_0.4_0.3_0.3",
]
batchSizes = [8, 16, 32]
models = ["yolo11n.pt", "yolo11s.pt", 
        "yolo11m.pt", 
        # "yolo11l.pt", "yolo11x.pt"
        ]

# 你的数据集根（每个数据集文件夹里应该有 swd_detection.yaml）
RUN_DATA_ROOT = Path("/workspace/models/runs_yolov11_det_insect/datasets")
# 训练输出根
PROJECT_ROOT = Path("/workspace/models/runs_yolov11_det_insect/outputs")

IMGSZ_INIT = 640      # 检测任务常用 640
EPOCHS = 1000          # 一般 150~300 就够；太长反而过拟合
WORKERS = 4
SEED = 42
# =========================


def find_yaml(folder: Path) -> Path:
    """在数据集文件夹下自动找 YAML。优先 swd_detection.yaml，其次唯一 *.yaml。"""
    if folder.is_file() and folder.suffix == ".yaml":
        return folder
    if not folder.exists():
        raise FileNotFoundError(f"数据集目录不存在: {folder}")

    y1 = folder / "swd_detection.yaml"
    if y1.exists(): 
        return y1

    yamls = list(folder.glob("*.yaml"))
    if len(yamls) == 1:
        return yamls[0]
    elif len(yamls) == 0:
        raise FileNotFoundError(f"找不到 YAML: {folder} 下没有 *.yaml")
    else:
        raise FileNotFoundError(f"找到多个 YAML，请保留一个或改名 swd_detection.yaml: {yamls}")


def has_nonempty_dir(p: Path) -> bool:
    return p.exists() and any(p.iterdir())


def train_one_combo(model_file: str, yaml_path: Path, batch: int):
    """带 OOM 自动降级重试：先降 batch，再降 imgsz。"""
    model = YOLO(model_file)
    current_bs = batch
    current_imgsz = IMGSZ_INIT

    run_name_base = f"{Path(model_file).stem}_{yaml_path.parent.name}_bs{batch}_sz{IMGSZ_INIT}"

    while True:
        print(f"\n🚀 Training model={model_file}  data={yaml_path}  batch={current_bs}  imgsz={current_imgsz}")
        try:
            model.train(
                data=str(yaml_path),
                epochs=EPOCHS,
                imgsz=current_imgsz,
                batch=current_bs,
                device=0,
                workers=WORKERS,
                seed=SEED,
                project=str(PROJECT_ROOT / Path(model_file).stem),
                name=f"{run_name_base}",
                # —— detection 任务的增强参数 —— #
                mosaic=0.5,              # 检测任务可以开启马赛克增强
                perspective=0.0005,
                fliplr=0.5,
                degrees=10.0,            # 检测任务可以角度增强更强
                scale=0.5,               # 检测任务可以缩放增强更强
                shear=2.0,
                # —— 早停/保存 —— #
                patience=30,           # 早停容忍
                save=True,
                save_period=25,        # 每隔 N epoch 存一次
                plots=True,
            )
            break  # 训练成功
        except RuntimeError as e:
            msg = str(e)
            if "CUDA out of memory" in msg:
                # 先减半 batch；如果已经 1，再降 imgsz（到不低于 640，32 对齐）
                if current_bs > 1:
                    current_bs = max(1, current_bs // 2)
                    print(f"⚠️  OOM，降 batch 重试：batch={current_bs}")
                    continue
                elif current_imgsz > 640:
                    current_imgsz = max(640, int(current_imgsz * 0.8 // 32 * 32))
                    print(f"⚠️  OOM，降 imgsz 重试：imgsz={current_imgsz}")
                    continue
                else:
                    print(f"⏭️  跳过该组合（仍 OOM）：{model_file} @ {yaml_path}")
                    return
            else:
                raise  # 其它错误直接抛出

    # —— 评测（有 test 就测 test，没有就测 val）——
    ds_root = yaml_path.parent
    test_dir = ds_root / "images" / "test"
    split = "test" if has_nonempty_dir(test_dir) else "val"
    print(f"🧪 Evaluating on split={split} ...")
    model.val(
        data=str(yaml_path),
        split=split,
        imgsz=current_imgsz,
        batch=min(current_bs, 16),
        project=str(PROJECT_ROOT / Path(model_file).stem),
        name=f"{run_name_base}_{split}",
    )


if __name__ == "__main__":
    PROJECT_ROOT.mkdir(parents=True, exist_ok=True)

    for modelFile in models:
        for dataFolderName in dataFolderNameList:
            yaml_path = find_yaml(RUN_DATA_ROOT / dataFolderName)
            for bs in batchSizes:
                train_one_combo(modelFile, yaml_path, bs)

    print("\n✅ All jobs finished.")


🚀 Training model=yolo11n.pt  data=/workspace/models/runs_yolov11_det_insect/datasets/insect_split_0.7_0.2_0.1/swd_detection.yaml  batch=8  imgsz=640
New https://pypi.org/project/ultralytics/8.3.218 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15818MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/workspace/models/runs_yolov11_det_insect/datasets/insect_split_0.7_0.2_0.1/swd_detection.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1000, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, k

train: Scanning /workspace/models/runs_yolov11_det_insect/datasets/insect_split_0.7_0.2_0.1/labels/train... 51 images, 0 backgrounds, 0 corrupt: 100%|██████████| 51/51 [00:00<00:00, 1639.79it/s]

train: New cache created: /workspace/models/runs_yolov11_det_insect/datasets/insect_split_0.7_0.2_0.1/labels/train.cache


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1501.5±1077.8 MB/s, size: 45.9 KB)


val: Scanning /workspace/models/runs_yolov11_det_insect/datasets/insect_split_0.7_0.2_0.1/labels/val... 14 images, 0 backgrounds, 0 corrupt: 100%|██████████| 14/14 [00:00<00:00, 2513.39it/s]

...


36 epochs completed in 0.008 hours.
Optimizer stripped from /workspace/models/runs_yolov11_det_insect/outputs/yolo11m/yolo11m_insect_split_0.4_0.3_0.3_bs32_sz6402/weights/last.pt, 40.5MB
Optimizer stripped from /workspace/models/runs_yolov11_det_insect/outputs/yolo11m/yolo11m_insect_split_0.4_0.3_0.3_bs32_sz6402/weights/best.pt, 40.5MB

Validating /workspace/models/runs_yolov11_det_insect/outputs/yolo11m/yolo11m_insect_split_0.4_0.3_0.3_bs32_sz6402/weights/best.pt...
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15818MiB)
YOLO11m summary (fused): 125 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 12.20it/s]


                   all         22         71      0.619      0.592      0.596      0.248
Speed: 0.1ms preprocess, 2.2ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /workspace/models/runs_yolov11_det_insect/outputs/yolo11m/yolo11m_insect_split_0.4_0.3_0.3_bs32_sz6402
🧪 Evaluating on split=test ...
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5080, 15818MiB)
YOLO11m summary (fused): 125 layers, 20,030,803 parameters, 0 gradients, 67.6 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2015.3±1206.7 MB/s, size: 47.2 KB)


val: Scanning /workspace/models/runs_yolov11_det_insect/datasets/insect_split_0.4_0.3_0.3/labels/test.cache... 23 images, 0 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.15it/s]


                   all         23         95      0.525      0.526      0.492      0.225
Speed: 0.1ms preprocess, 5.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /workspace/models/runs_yolov11_det_insect/outputs/yolo11m/yolo11m_insect_split_0.4_0.3_0.3_bs32_sz640_test

✅ All jobs finished.
